In [11]:
import torch
from torchvision import models, transforms
from PIL import Image
from tqdm.notebook import tqdm
import os
import cv2

class FeatureExtractor:
    def __init__(self):
        self.model = models.vgg19(weights='IMAGENET1K_V1')
        self.max_pooling_indices = [9, 18, 27, 36]
        self.model.eval()
        
    @torch.no_grad()
    def extract_feature_vector(self, x, stage=0, verbose=False):
        for i, feature in enumerate(self.model.features):
            if verbose:
                print(f'Passing through layer {type(feature)}')
            if i == self.max_pooling_indices[stage]:
                return x
            x = feature(x)

def resize_image(image, size):
    h,w,c = image.shape

    if h > w:
        w = int(size*w/h)
        h = size
    else:
        h = int(size*h/w)
        w = size
    new_image = np.zeros((size, size, 3), dtype=np.uint8)

    image = cv2.resize(image, (w, h))
    new_image[0:h, 0:w, :] = image
    return new_image


In [45]:
## TRANSFORMAR IMAGE DATASET A VECTORES DE LA VGG

feature_extractor = FeatureExtractor()
backbone_transforms = transforms.Compose([transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
fruits_dataset_path = './data/fruits_dataset/'
labels = []
feature_vectors = []
for label_name in tqdm(os.listdir(fruits_dataset_path)):
    if label_name == '.DS_Store':
        continue
    folder_path = os.path.join(fruits_dataset_path, label_name)
    for image_name in tqdm(os.listdir(folder_path), leave=False):
        image_path = os.path.join(folder_path, image_name)
        image = Image.open(image_path)
        if image.mode in ['RGBA', 'L', 'P']:
            image = image.convert('RGB')
        image = np.array(image)[...,::-1]
        image = resize_image(image, 255)
        transformed_image = backbone_transforms(image).unsqueeze(0)
        feature_vectors += [feature_extractor.extract_feature_vector(transformed_image, 3).flatten().numpy()]
        labels += [label_name]
        

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [50]:
fv_dataset = np.array(feature_vectors)
labels = np.array(labels)
labels2 = np.unique(labels, return_inverse=True)[1]

In [41]:
image = Image.open(image_path)
if image.mode in ['RGBA', 'L']:
    image = image.convert('RGB')

In [44]:
image.mode

'P'

In [3]:
from PIL import Image
from torchvision import transforms

image = Image.fromarray(np.zeros((255,255,3), dtype=np.uint8))
backbone_transforms(image)

NameError: name 'np' is not defined

In [ ]:
backbone_transforms(image)

In [6]:
fe = FeatureExtractor()

In [11]:
fe.extract_feature_vector(image,stage=3, verbose=True).shape

Passing through layer <class 'torch.nn.modules.conv.Conv2d'>
Passing through layer <class 'torch.nn.modules.activation.ReLU'>
Passing through layer <class 'torch.nn.modules.conv.Conv2d'>
Passing through layer <class 'torch.nn.modules.activation.ReLU'>
Passing through layer <class 'torch.nn.modules.pooling.MaxPool2d'>
Passing through layer <class 'torch.nn.modules.conv.Conv2d'>
Passing through layer <class 'torch.nn.modules.activation.ReLU'>
Passing through layer <class 'torch.nn.modules.conv.Conv2d'>
Passing through layer <class 'torch.nn.modules.activation.ReLU'>
Passing through layer <class 'torch.nn.modules.pooling.MaxPool2d'>
Passing through layer <class 'torch.nn.modules.conv.Conv2d'>
Passing through layer <class 'torch.nn.modules.activation.ReLU'>
Passing through layer <class 'torch.nn.modules.conv.Conv2d'>
Passing through layer <class 'torch.nn.modules.activation.ReLU'>
Passing through layer <class 'torch.nn.modules.conv.Conv2d'>
Passing through layer <class 'torch.nn.modules.ac

torch.Size([1, 512, 15, 15])

In [3]:
models.vgg19(weights='IMAGENET1K_V1')

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [14]:
fvs = []
labels = []
for image in range(10):
    image = torch.zeros((1, 3, 255, 255)) ## IMAGEN
    fv = fe.extract_feature_vector(image,stage=3, verbose=False).flatten().numpy()
    fvs += [fv]
    labels += [label]

In [16]:
import numpy as np
np.array(fvs).shape

(10, 115200)